In [1]:
exec(open("cleanup.py").read())

In [2]:
import json
from collections import defaultdict

def get_response(fname):
    with open(fname, 'r') as f:
        data = f.read()

    response = json.loads(data)

    surv_resp = defaultdict(dict)
    pass_resp = defaultdict(str)
    time_resp = defaultdict(str)

    for k, v in response.items():
        if 'response' in k:
            temp = k.split("_")
            pid = temp[0].strip('p')
            v = json.loads(v)
            surv_resp[pid].update(v)
        elif 'time' in k:
            temp = k.split("_")
            pid = temp[0]
            time_resp[pid] = float(v)
        elif 'password' in k:
            pass_resp[k] = v

    return (pass_resp, time_resp, surv_resp)

In [3]:
page_map = {
    '0': 'startstudy',
    '1': 'fpass',
    '2': 'fcog',
    '3': 'fpasssurvey',
    '4': 'fpassrecall',
    '5': 'pagegif',
    '6': 'desinint',
    '7': 'quest',
    '8': 'csquest',
    '9': 'motivation',
    '10': 'spass',
    '11': 'scog',
    '12': 'spasssurvey',
    '13': 'spassrecall',
    '14': 'demo',
    '15': 'thanks',
}

In [4]:
import os
fpath = './responses'
password_response = []
time_response = []
survey_response = []
for i, fname in enumerate(os.listdir(fpath)):
    pss, tim, sur = get_response(os.path.join(fpath, fname))
    password_response.append(pss)
    time_response.append(tim)
    survey_response.append(sur)

In [5]:
import pandas as pd
pd.DataFrame(password_response)

,password2,password1,password1_recall1,password2_recall1
0,ee,dd,ss,NaN
1,as,as,NaN,asd


In [6]:
page_time = pd.DataFrame(time_response)
page_time

,2,12,5,10,8,0,7,6,11,9,4,1,3
0,3.833,4.681,2.035,2.082,235.582,2.333,27.935,8.187,19.620,24.915,6.517,3.476,10.734
1,3.492,9.900,7.382,3.895,18.600,15.179,72.818,9.196,348.595,25.557,4.400,12.138,18.796


In [7]:
# top 5 pages with higher time
top5 = pd.DataFrame(columns=['page', 'time'])
for i, data in page_time.mean().sort_values(ascending=False).head(5).items():
    top5 = top5.append(pd.DataFrame([[page_map.get(i), data]], columns=top5.columns), ignore_index=True)

top5

,page,time
0,scog,184.1075
1,csquest,127.0910
2,quest,50.3765
3,motivation,25.2360
4,fpasssurvey,14.7650


In [8]:
pd.DataFrame(survey_response)

,7,3,8,13,11
0,"{'1-26': '4', '1-23': '4', '1-28': '5', '1-21'...","{'1-1': '4', '2-1': '3', '2-2': '3', '2-3': '2...","{'3-1': '3', '4-1': '3', '5-1': '2', '6-1': ['...","{'1-1': 'Non-Binary', '2-1': '35-39 years old'...","{'1-1': '6', '2-1': '5', '2-2': '4', '2-3': '4..."
1,"{'1-28': '7', '1-26': '7', '1-25': '7', '1-27'...","{'1-1': '6', '2-1': '6', '2-2': '6', '2-3': '5...","{'1-1': '7', '1-2': '6', '1-3': '7', '1-4': '6...","{'1-1': 'Man', '2-1': 'I prefer not to answer'...","{'1-1': '6', '2-1': '5', '2-2': '5', '2-3': '3..."
